In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import glob
import motes_corpus.youtube as yt
import json
import pandas as pd
import spacy
import numpy as np

In [5]:
caption_paths = glob.glob('/data/motes/yt_captions/**/*')
# Example
caption = yt.YTCaption(caption_paths[2])
caption.text[:200] + '...'

"The special weapons I've developed will assure our victory. I wish I had a device that could locate your lost brother Vrak. Leave it to me. I've been working on something big. (button beeping) It's wa..."

In [8]:
all_text = ""
for path in caption_paths:
    try:
        caption = yt.YTCaption(path)
        all_text += '\n' + caption.text
    except KeyboardInterrupt:
        raise
    except json.JSONDecodeError:
        print('Invalid JSON for {}. Check or delete.'.format(path))
    except:
        raise
        print('Error with {}'.format(path))

### Number of words in the dataset (so far)

In [9]:
len(all_text.split())

4450218

### Top Words

In [10]:
nlp = spacy.load('en_core_web_sm')

In [18]:
all_words = []
for x in all_text.split('\n'):
    doc = nlp(x, disable=['parser', 'tagger','ner','textcat'])
    all_words += [word.lemma_.lower() for word in doc if (not word.is_stop) and word.is_alpha]
top_words = pd.Series(all_words).value_counts()
print(" ".join(top_words.head(500).index.tolist()))

go like oh let know get look come right okay time music think want yes to yeah need good find thing little way hey day help play goo guy say school work kid dude ready power love big great laugh start zz try dad thank student new mean wait today tell god mom well use people ha friend year lot whoa sure man baby feel kind water talk eat watch ranger learn rangers leave fun uh stop place maybe make turn cool monster blue game run hello color hear super happen wow long home world red take different hand video away sound live hi ask house check nice word alright see open call job bite question idea ah actually happy high child grunt ninja family cat huh life build bring line fast numb light end fire um sorry head inside remember green boy ronald district door thanks favorite fly close hold room book stay write bye give guess story food win read ok nick problem miss car hard meet pretty star awesome believe tree bad halloween will hope change teacher eye old small kids jump earth night week

In [19]:
top_words.head(500)

go            24106
like          23063
oh            21503
let           18782
know          15189
              ...  
destroy         697
cry             697
strike          697
lead            695
definitely      695
Length: 500, dtype: int64

### By TFIDF

In [22]:
idf = pd.read_parquet('/projects/creativity-aut/data/idf-vals.parquet').set_index('token').to_dict()['IPF']
top_wordsdf = top_words.reset_index().rename(columns={'index':'token', 0:'count'})
top_wordsdf['idf'] = top_wordsdf.token.apply(lambda x: idf[x] if x in idf else None)
top_wordsdf['tf-idf'] = top_wordsdf['count'] * top_wordsdf['idf']
top_wordsdf['wf-idf'] = (top_wordsdf['count'].apply(np.log) + 1) * top_wordsdf['idf']
top_wordsdf.sort_values('tf-idf', ascending=False).iloc[:20]

,token,count,idf,tf-idf,wf-idf
9,okay,12776,4.339616,55442.933340,45.372089
2,oh,21503,1.904725,40957.309426,20.906166
16,yeah,8929,4.228069,37752.430407,42.691068
33,dude,5833,3.753486,21894.084721,36.301041
27,goo,6293,3.380653,21274.451133,32.951881
0,go,24106,0.879535,21202.065593,9.754231
23,hey,6920,2.950612,20418.232266,29.040424
41,zz,5071,4.003710,20302.813575,38.160535
28,guy,6227,3.250274,20239.456989,31.646781
3,let,18782,1.020807,19172.802543,11.066219


In [24]:
print("\t".join(top_wordsdf.sort_values('wf-idf', ascending=False).iloc[:200].token.tolist()))

ronald	halloween	zord	brody	jenna	ninja	okay	chloe	superhero	backpack	morphin	yummy	freddy	zombie	yeah	upbeat	butterbean	noah	woah	guppy	christmas	activate	karina	gaga	ahhh	hmm	alright	website	oink	peppa	pbs	ricky	nickelodeon	whoosh	maggie	chris	jesus	whoa	google	download	krishna	steve	dino	ooh	hayley	corina	preston	starburst	daniel	zz	pihu	berby	zap	gosei	frankie	zordon	uppercase	elinor	spaceship	dinosaur	mommy	yay	gotcha	tyler	cupcake	zeo	smoothie	dude	beep	cindy	pizza	aviva	canticos	calvin	wednesday	robot	ninjas	olympia	chacha	johny	hemet	morph	mickey	zoom	yep	pandemic	erika	wanna	mcdonald	ahh	scary	elmo	sabertooth	vroom	oooh	recycle	sydney	molly	goldie	teddy	elliot	sarah	lizzy	firefighter	chuchu	googoo	ryou	elijah	robotic	jurassic	california	jake	rangers	jodi	shh	katie	america	oswald	headphone	abby	hahaha	andy	yumi	hello	suspenseful	devon	aww	meatball	itsy	saturday	superpower	uranus	meow	stinky	googly	timmy	oops	wow	sunspot	ribbit	titano	zuma	astronaut	uh	danny	awesome	kaden	odius	